In [14]:
import pandas as pd
import pulp

In [15]:
data = pd.read_csv('./app/data/elements.csv')

In [5]:
data.head()

,id,name,element_type_name,team_name,total_points,now_cost
0,330,Harry Maguire,DEF,Man Utd,69,50
1,344,Anthony Elanga,MID,Man Utd,59,50
2,341,Aaron Wan-Bissaka,DEF,Man Utd,41,45
3,331,Frederico Rodrigues de Paula Santos,MID,Man Utd,93,55
4,343,Brandon Williams,DEF,Man Utd,36,40


In [16]:
# const
# Position name
POSITIONS = ['GKP', 'DEF', 'MID', 'FWD']
# Limit on the number of players by position
PLAYER_LIMIT_BY_POSITIONS = {'GKP': 2, 'DEF': 5, 'MID': 5, 'FWD': 3}
# Limit on the number of players
TOTAL_PLAYERS = sum([p for p in PLAYER_LIMIT_BY_POSITIONS.values()])
# Limit on the number of players on the same team
SAME_TEAM_LIMIT = 3
# Limit on cost
COST_LIMIT = 970
# favorit team （default: none）
# FAVORIT_TEAM = 'Man Utd'
FAVORIT_TEAMS = ['Man Utd']
# EXCLUSIVE TEAMS
EXCLUSIVE_TEAMS = ['West Ham']

In [24]:
fun = lambda x: pulp.LpVariable(f'{x.id}_{x.element_type_name}_{x.team_name}', cat='Binary')
data['variables'] = list(data.apply(fun, axis=1))

In [31]:
prob = pulp.LpProblem('fpl_planner', sense = pulp.LpMaximize)
# ポジションごとの人数の制約
for p in data.element_type_name.unique():
    dt = data[data.element_type_name == p]
    prob += pulp.lpSum(dt.variables) == PLAYER_LIMIT_BY_POSITIONS[p]

In [37]:
# 同じチーム 3 人の制約
for t in data.team_name.unique():
    dt = data[data.team_name == t]
    prob += pulp.lpSum(dt.variables) <= SAME_TEAM_LIMIT

# お気に入りチーム選手を絶対一人入れる
for t in FAVORIT_TEAMS:
    dt = data[data.team_name == t]
    prob += pulp.lpSum(dt.variables) >= 1

# 除外するチーム
for t in EXCLUSIVE_TEAMS:
    dt = data[data.team_name == t]
    prob += pulp.lpSum(dt.variables) == 0

In [42]:
# 目的関数
prob += pulp.lpDot(data.total_points, data.variables)
# コストの制限
prob += pulp.lpDot(data.now_cost, data.variables) <= COST_LIMIT
# 全体の選手数の制限
prob += pulp.lpSum(data.variables) == TOTAL_PLAYERS

/Users/nori/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pulp/pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [46]:
print(prob)

fpl_planner:
MAXIMIZE
44*101_MID_Brighton + 2*102_GKP_Brighton + 89*103_FWD_Brighton + 88*104_MID_Brighton + 48*105_DEF_Brighton + 92*106_DEF_Brighton + 61*107_MID_Brighton + 59*108_DEF_Brighton + 95*109_DEF_Brighton + 107*10_DEF_Arsenal + 98*110_FWD_Brighton + 141*111_MID_Brighton + 126*112_DEF_Brighton + 126*113_GKP_Brighton + 74*114_DEF_Brighton + 13*115_MID_Brighton + 93*116_MID_Brighton + 58*117_MID_Brighton + 58*118_MID_Brighton + 5*119_MID_Brighton + 55*11_FWD_Arsenal + 27*120_MID_Brighton + 2*121_MID_Brighton + 87*127_DEF_Chelsea + 130*128_DEF_Chelsea + 128*129_DEF_Chelsea + 125*12_MID_Arsenal + 106*130_MID_Chelsea + 14*131_MID_Chelsea + 76*132_MID_Chelsea + 19*133_GKP_Chelsea + 79*134_MID_Chelsea + 79*135_MID_Chelsea + 52*136_MID_Chelsea + 62*137_FWD_Chelsea + 4*138_DEF_Chelsea + 55*139_DEF_Chelsea + 179*13_MID_Arsenal + 77*140_MID_Chelsea + 99*141_DEF_Chelsea + 169*142_MID_Chelsea + 30*143_DEF_Chelsea + 49*144_MID_Chelsea + 112*145_FWD_Chelsea + 141*146_DEF_Chelsea + 130*147_

In [49]:
status = prob.solve()
print("Status", pulp.LpStatus[status])
print(prob.objective.value())

Status Optimal
2383.0


In [51]:
data['flag'] = data.apply(lambda x: x.variables.value(), axis=1)
data[data.flag == 1].sort_values('element_type_name')

,id,name,element_type_name,team_name,total_points,now_cost,variables,flag
55,16,Gabriel dos Santos Magalhães,DEF,Arsenal,146,50,16_DEF_Arsenal,1.0
128,43,Matty Cash,DEF,Aston Villa,147,50,43_DEF_Aston_Villa,1.0
197,276,Joel Matip,DEF,Liverpool,170,60,276_DEF_Liverpool,1.0
200,285,Trent Alexander-Arnold,DEF,Liverpool,208,75,285_DEF_Liverpool,1.0
297,306,João Cancelo,DEF,Man City,201,70,306_DEF_Man_City,1.0
129,40,Ollie Watkins,FWD,Aston Villa,131,75,40_FWD_Aston_Villa,1.0
321,95,Bryan Mbeumo,FWD,Brentford,119,60,95_FWD_Brentford,1.0
323,80,Ivan Toney,FWD,Brentford,139,70,80_FWD_Brentford,1.0
485,327,David De Gea Quintana,GKP,Man Utd,132,50,327_GKP_Man_Utd,1.0
502,281,Alisson Ramses Becker,GKP,Liverpool,176,55,281_GKP_Liverpool,1.0
